In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from data_import import (
    load_data, load_ecb_1y_yield,
    fill_liabilities, drop_high_leverage_firms,
    prepare_merton_inputs
)

print(Path.cwd())

c:\Users\afons\OneDrive\Desktop\ESE\FCS\Merton_NIGbayesian


In [18]:
# data loading and initial processing
ret_daily, bs, coverage = load_data(
    Path.cwd() / "Jan2025_Accenture_Dataset_ErasmusCase.xlsx",
    start_date="2012-01-01",
    end_date="2025-12-19",
    enforce_coverage=True,
    coverage_tol=0.95,
    liabilities_scale="auto",
    verbose=True,
)

df_rf = load_ecb_1y_yield(
    startPeriod="2010-01-01",
    endPeriod="2025-12-31",
    out_file="ecb_yc_1y_aaa.xml",
    verify_ssl=True,  # recommended if it works
)

df_cal = ret_daily[["date"]].drop_duplicates().sort_values("date").reset_index(drop=True)

debt_daily = fill_liabilities(bs, df_cal)

ret_filt, bs_filt, lev_by_firm, dropped = drop_high_leverage_firms(
    ret_daily,
    bs,
    df_calendar=df_cal,
    debt_daily=debt_daily,
    lev_threshold=8.0,
    lev_agg="median",
    verbose=True,
)

# keep debt panel consistent with filtered firms
keep = set(ret_filt["gvkey"].astype(str).unique())
debt_daily_filt = debt_daily[debt_daily["gvkey"].astype(str).isin(keep)].copy()

# Merton
merton_df = prepare_merton_inputs(ret_filt, bs_filt, df_rf, debt_daily=debt_daily_filt)

[load_data] Firms (ret_daily): 46
[load_data] Date range (ret_daily): 2012-01-03 .. 2025-12-19
[load_data] Coverage min/median/max: 0.999 / 1.000 / 1.000
[load_data] liabilities_scale_used: 1e+06
[load_data] QA mcap_reported<=0 rows (raw windowed mkt): 62
Data has been written to ecb_yc_1y_aaa.xml
[drop_high_leverage_firms] agg=median, threshold=8.0
[drop_high_leverage_firms] firms before: 46 | after: 36
[drop_high_leverage_firms] dropped firms: 10


In [19]:
# Check missingness of sigma_E
print("Missing sigma_E %:", merton_df["sigma_E"].isna().mean() * 100)
print(merton_df["sigma_E"].describe())

# Check missingness of B and r
print("Missing B %:", merton_df["B"].isna().mean() * 100)
print("Missing r %:", merton_df["r"].isna().mean() * 100)

Missing sigma_E %: 3.431054858754908
count    126655.000000
mean          0.254476
std           0.083942
min           0.117892
25%           0.195147
50%           0.238274
75%           0.292771
max           0.743236
Name: sigma_E, dtype: float64
Missing B %: 0.0
Missing r %: 0.0


In [20]:
# BUILDING THE CALIBRATION DATASET DROPPING ROWS WITH MISSING INPUTS
df = merton_df.copy()

# first date where B becomes available for each firm
first_B_date = (
    df.dropna(subset=["B"])
      .groupby("gvkey")["date"]
      .min()
      .rename("first_B_date")
)
# first date where sigma_E becomes available for each firm
first_sigma_date = (
    df.dropna(subset=["sigma_E"])
      .groupby("gvkey")["date"]
      .min()
      .rename("first_sigma_date")
)

starts = pd.concat([first_B_date, first_sigma_date], axis=1)
starts["calib_start"] = starts[["first_B_date","first_sigma_date"]].max(axis=1)

# attach and filter
df2 = df.merge(starts["calib_start"], on="gvkey", how="left")

calib = (
    df2[df2["date"] >= df2["calib_start"]]
      .dropna(subset=["E","B","r","sigma_E"])
      .query("E > 0 and B > 0")
      .copy()
      .rename(columns={"B":"B_drop"})
)

print("Rows before:", len(df), "Rows after firm-specific start + required inputs:", len(calib))
print("Dropped %:", (len(df)-len(calib))/len(df))
print("Missing values in calibration dataset:")
print(calib.isna().sum())

calib_drop = calib.copy()

Rows before: 131155 Rows after firm-specific start + required inputs: 126655
Dropped %: 0.03431054858754908
Missing values in calibration dataset:
gvkey            0
date             0
E                0
logret_mcap      0
isin             0
company          0
country_iso      0
r                0
B_drop           0
sigma_E_daily    0
sigma_E          0
calib_start      0
dtype: int64


In [ ]:
# Run the Merton model estimation of sigma  (and build weekly results_df for PDs)

from merton_model_afonso import calibrate_sigmaV_window_weekly_merton

start = pd.Timestamp("2012-01-01")
end   = pd.Timestamp("2013-12-31")

n = 2
gvkeys = sorted(merton_df["gvkey"].astype(str).unique())[:n]

results = {}
rows = []           # firm-level summary (like before)
weekly_rows = []    # NEW: weekly long-format rows for PD inputs

for gvkey in gvkeys:
    # subset firm + window
    df_firm = merton_df[merton_df["gvkey"].astype(str) == gvkey].copy()
    df_firm["date"] = pd.to_datetime(df_firm["date"])
    df_firm = df_firm[df_firm["date"].between(start, end)].copy()

    # KMV-style horizon
    df_firm["T"] = 1.0

    # required numeric columns
    df_firm["B_used"] = pd.to_numeric(df_firm["B"], errors="coerce")
    df_firm["E"] = pd.to_numeric(df_firm["E"], errors="coerce")
    df_firm["r"] = pd.to_numeric(df_firm["r"], errors="coerce")

    # keep only inputs needed for inversion (sigma_E optional)
    df_firm = (
        df_firm.dropna(subset=["date", "E", "B_used", "r", "T"])
               .query("E > 0 and B_used > 0")
               .sort_values("date")
    )

    print("\n" + "="*70)
    print("gvkey:", gvkey)
    if "company" in df_firm.columns and len(df_firm):
        print("company:", df_firm["company"].iloc[0])
    print("Daily rows used:", len(df_firm))
    if len(df_firm):
        print("date range:", df_firm["date"].min().date(), "to", df_firm["date"].max().date())

    if len(df_firm) < 10:
        print("Skipping: not enough daily rows after cleaning.")
        rows.append({"gvkey": gvkey, "ok": False, "msg": "too_few_daily_rows"})
        continue

    sigmaV_hat, weekly_df, ok, msg = calibrate_sigmaV_window_weekly_merton(
        df_firm,
        week_ending="W-FRI",
        ann_factor=52.0,
        sigmaV_init=None,
        E_col="E",
        B_col="B_used",
        r_col="r",
        T_col="T",
        sigmaE_col="sigma_E",
    )

    print("ok:", ok)
    print("msg:", msg)
    print("sigma_V_hat (annual):", sigmaV_hat)
    print("weekly obs used:", 0 if weekly_df is None else len(weekly_df))

    # store outputs (unchanged)
    results[gvkey] = {"sigmaV_hat": sigmaV_hat, "weekly_df": weekly_df, "ok": ok, "msg": msg}
    rows.append({
        "gvkey": gvkey,
        "ok": ok,
        "msg": msg,
        "sigma_hat": sigmaV_hat,  # NOTE: renamed for consistency with your request
        "n_daily": len(df_firm),
        "n_weekly": 0 if weekly_df is None else len(weekly_df),
    })

    # NEW: add weekly rows needed for PDs (only if ok)
    if ok and weekly_df is not None and len(weekly_df):
        # liability from LAST DAY of training window (last available daily in df_firm)
        B_end = float(df_firm["B_used"].iloc[-1])

        w = weekly_df[["date", "V_hat", "dlogV", "r"]].copy()
        w = w.rename(columns={"dlogV": "d_V_hat", "r": "risk_free"})
        w["gvkey"] = gvkey
        w["sigma_hat"] = float(sigmaV_hat)
        w["B_end"] = B_end

        # keep exactly the columns you requested
        w = w[["gvkey", "date", "sigma_hat", "V_hat", "d_V_hat", "risk_free", "B_end"]]
        weekly_rows.append(w)

    # optional: peek per firm (comment out if too verbose)
    if weekly_df is not None and len(weekly_df):
        display(weekly_df.head(3), weekly_df.tail(3))

# firm-level summary (like your old results_df)
summary_df = pd.DataFrame(rows).sort_values("gvkey").reset_index(drop=True)

# NEW: long weekly df with everything needed for PDs + mu_hat
results_df = (
    pd.concat(weekly_rows, ignore_index=True)
      .sort_values(["gvkey", "date"])
      .reset_index(drop=True)
)

summary_df, results_df


gvkey: 100022
company: BAYERISCHE MOTOREN WERKE AKT
Daily rows used: 521
date range: 2012-01-03 to 2013-12-31
ok: True
msg: converged(it=1)
sigma_V_hat (annual): 0.08481598649165671
weekly obs used: 105


,date,E,B,r,T,V_hat,d1,d2,logV,dlogV,sigma_V_win,window_ok,window_msg
0,2012-01-06,3.344083e+10,8.576700e+10,0.001873,1.0,1.190472e+11,3.930384,3.845569,25.502786,NaN,0.084816,True,converged(it=1)
1,2012-01-13,3.518662e+10,8.576700e+10,0.001492,1.0,1.208257e+11,4.100724,4.015909,25.517615,0.014829,0.084816,True,converged(it=1)
2,2012-01-20,3.828087e+10,8.576700e+10,0.002094,1.0,1.238685e+11,4.401063,4.316248,25.542486,0.024871,0.084816,True,converged(it=1)


,date,E,B,r,T,V_hat,d1,d2,logV,dlogV,sigma_V_win,window_ok,window_msg
102,2013-12-20,5.056158e+10,1.014480e+11,0.001288,1.0,1.518790e+11,4.815446,4.730631,25.746350,0.016899,0.084816,True,converged(it=1)
103,2013-12-27,5.142243e+10,1.014480e+11,0.001213,1.0,1.527475e+11,4.881788,4.796973,25.752052,0.005702,0.084816,True,converged(it=1)
104,2013-12-31,5.130203e+10,1.014480e+11,0.000942,1.0,1.526546e+11,4.871415,4.786600,25.751443,-0.000608,0.084816,True,converged(it=1)



gvkey: 100080
company: BAYER AG
Daily rows used: 521
date range: 2012-01-03 to 2013-12-31
ok: True
msg: converged(it=1)
sigma_V_hat (annual): 0.12667071789878853
weekly obs used: 105


,date,E,B,r,T,V_hat,d1,d2,logV,dlogV,sigma_V_win,window_ok,window_msg
0,2012-01-06,4.281109e+10,3.254300e+10,0.001873,1.0,7.529319e+10,6.700233,6.573562,25.044656,NaN,0.126671,True,converged(it=1)
1,2012-01-13,4.367938e+10,3.254300e+10,0.001492,1.0,7.617388e+10,6.789025,6.662355,25.056284,0.011629,0.126671,True,converged(it=1)
2,2012-01-20,4.425825e+10,3.254300e+10,0.002094,1.0,7.673317e+10,6.851533,6.724862,25.063600,0.007316,0.126671,True,converged(it=1)


,date,E,B,r,T,V_hat,d1,d2,logV,dlogV,sigma_V_win,window_ok,window_msg
102,2013-12-20,8.224823e+10,3.276700e+10,0.001288,1.0,1.149731e+11,9.983259,9.856588,25.467964,0.029463,0.126671,True,converged(it=1)
103,2013-12-27,8.521697e+10,3.276700e+10,0.001213,1.0,1.179443e+11,10.184089,10.057418,25.493478,0.025514,0.126671,True,converged(it=1)
104,2013-12-31,8.430733e+10,3.276700e+10,0.000942,1.0,1.170435e+11,10.121424,9.994753,25.485811,-0.007667,0.126671,True,converged(it=1)


(    gvkey    ok              msg  sigma_hat  n_daily  n_weekly
 0  100022  True  converged(it=1)   0.084816      521       105
 1  100080  True  converged(it=1)   0.126671      521       105,
       gvkey       date  sigma_hat         V_hat   d_V_hat  risk_free  \
 0    100022 2012-01-06   0.084816  1.190472e+11       NaN   0.001873   
 1    100022 2012-01-13   0.084816  1.208257e+11  0.014829   0.001492   
 2    100022 2012-01-20   0.084816  1.238685e+11  0.024871   0.002094   
 3    100022 2012-01-27   0.084816  1.244749e+11  0.004884   0.002112   
 4    100022 2012-02-03   0.084816  1.277593e+11  0.026044   0.001579   
 ..      ...        ...        ...           ...       ...        ...   
 205  100080 2013-12-06   0.126671  1.121279e+11 -0.016177   0.001051   
 206  100080 2013-12-13   0.126671  1.116351e+11 -0.004404   0.001452   
 207  100080 2013-12-20   0.126671  1.149731e+11  0.029463   0.001288   
 208  100080 2013-12-27   0.126671  1.179443e+11  0.025514   0.001213   
 209

In [ ]:
# PD calculation

from merton_pd_afonso import (
    estimate_mu_from_weekly_implied_assets,
    merton_pd_rn_1y,
    merton_pd_physical_1y,
)

pd_rows = []

# iterate firms using summary_df (firm-level ok/msg)
for _, row in summary_df.iterrows():
    gvkey = str(row["gvkey"])
    ok = bool(row.get("ok", True))

    if not ok:
        pd_rows.append({"gvkey": gvkey, "ok": False, "msg": row.get("msg", "not_ok")})
        continue

    # pull this firm's weekly series from results_df
    g = results_df[results_df["gvkey"].astype(str) == gvkey].sort_values("date")

    if g.empty:
        pd_rows.append({"gvkey": gvkey, "ok": False, "msg": "missing_weekly_rows_in_results_df"})
        continue

    sigmaV_hat = float(g["sigma_hat"].iloc[0])
    B_end = float(g["B_end"].iloc[0])

    # PD date = last Friday (last weekly point)
    pd_date = pd.Timestamp(g["date"].iloc[-1])
    V_pd = float(g["V_hat"].iloc[-1])
    r_pd = float(g["risk_free"].iloc[-1])

    # estimate mu from weekly implied assets (needs a df with column 'dlogV')
    mu_hat = estimate_mu_from_weekly_implied_assets(
        pd.DataFrame({"dlogV": g["d_V_hat"].to_numpy(dtype=float)}),
        sigmaV_hat,
        ann_factor=52.0
    )

    PD_Q_1y = float(merton_pd_rn_1y(V_pd, B_end, r_pd, sigmaV_hat))
    PD_P_1y = float(merton_pd_physical_1y(V_pd, B_end, mu_hat, sigmaV_hat)) if np.isfinite(mu_hat) else np.nan

    pd_rows.append({
        "gvkey": gvkey,
        "ok": True,
        "sigma_hat": sigmaV_hat,
        "mu_hat": mu_hat,
        "pd_date_last_fri": pd_date,
        "V_pd": V_pd,
        "r_pd": r_pd,
        "B_end": B_end,
        "PD_Q_1y": PD_Q_1y,
        "PD_P_1y": PD_P_1y,
    })

pd_results = pd.DataFrame(pd_rows).sort_values("gvkey").reset_index(drop=True)
pd_results

,gvkey,ok,sigma_hat,mu_hat,pd_date_last_fri,V_pd,r_pd,B_end,PD_Q_1y,PD_P_1y
0,100022,True,0.084816,0.127925,2013-12-31,1.526546e+11,0.000942,1.014480e+11,8.483668e-07,1.652843e-10
1,100080,True,0.126671,0.228601,2013-12-31,1.170435e+11,0.000942,3.276700e+10,8.034381e-24,2.146021e-32


In [23]:
# OOS asset path

from merton_model_afonso import build_weekly_calendar_from_panel, invert_asset_one_week_merton

# settings
week_ending = "W-FRI"
T_horizon = 1.0          # KMV-style
n_weeks_oos = 13

# assumes these exist from previous cells:
# - merton_df  (daily panel)
# - summary_df (firm-level; has gvkey, ok, sigma_hat)
# - results_df (weekly in-sample; has gvkey, date, V_hat)

oos_rows = []

# use the same firms as before (first n) OR reuse the gvkeys list if you still have it
gvkeys = summary_df.loc[summary_df["ok"] == True, "gvkey"].astype(str).tolist()

# training window end date
train_end = pd.Timestamp("2013-12-31")
oos_start = train_end + pd.Timedelta(days=1)
oos_end   = train_end + pd.Timedelta(weeks=n_weeks_oos + 2)  # small buffer to ensure 13 week-ends exist

# map sigma_hat and last in-sample V_hat (warm start)
sigma_by_gv = summary_df.set_index(summary_df["gvkey"].astype(str))["sigma_hat"].to_dict()
lastV_by_gv = (
    results_df.sort_values(["gvkey", "date"])
              .groupby(results_df["gvkey"].astype(str))["V_hat"]
              .last()
              .to_dict()
)

for gvkey in gvkeys:
    sigma_hat = float(sigma_by_gv.get(gvkey, np.nan))
    if (not np.isfinite(sigma_hat)) or sigma_hat <= 0:
        print(f"[skip] gvkey={gvkey}: missing/invalid sigma_hat")
        continue

    # ---- daily out-of-sample slice ----
    g = merton_df[merton_df["gvkey"].astype(str) == gvkey].copy()
    g["date"] = pd.to_datetime(g["date"])
    g = g[g["date"].between(oos_start, oos_end)].copy()

    if g.empty:
        print(f"[skip] gvkey={gvkey}: no daily data in OOS range")
        continue

    # required columns
    g["E"] = pd.to_numeric(g["E"], errors="coerce")
    g["B_used"] = pd.to_numeric(g["B"], errors="coerce")   # assumes B already in correct units
    g["r"] = pd.to_numeric(g["r"], errors="coerce")
    g["T"] = T_horizon

    g = (
        g.dropna(subset=["date", "E", "B_used", "r", "T"])
         .query("E > 0 and B_used > 0")
         .sort_values("date")
    )

    if g.empty:
        print(f"[skip] gvkey={gvkey}: no usable rows after cleaning")
        continue

    # weekly dates (last trading day each week) and take first 13
    weekly_dates = build_weekly_calendar_from_panel(g, week_ending=week_ending)
    weekly_dates = weekly_dates[:n_weeks_oos]

    if len(weekly_dates) < n_weeks_oos:
        print(f"[warn] gvkey={gvkey}: only {len(weekly_dates)} weekly dates available (wanted {n_weeks_oos})")

    # index daily for fast lookup (dedupe dates)
    g_idx = g.groupby("date", as_index=False).last().set_index("date")

    # warm start from last in-sample V_hat (if available)
    V_prev = lastV_by_gv.get(gvkey, None)
    V_prev = float(V_prev) if (V_prev is not None and np.isfinite(V_prev) and V_prev > 0) else None

    # invert assets week by week
    for d in weekly_dates:
        if d not in g_idx.index:
            continue
        row = g_idx.loc[d]
        E_obs = float(row["E"])
        B_obs = float(row["B_used"])
        r_obs = float(row["r"])
        T_obs = float(row["T"])

        V_hat, d1, d2 = invert_asset_one_week_merton(
            E_obs, B_obs, r_obs, T_obs, sigma_hat,
            V_prev=V_prev,
            tol=1e-6,
            maxiter=200
        )
        V_prev = V_hat

        oos_rows.append({
            "gvkey": gvkey,
            "date": pd.Timestamp(d),
            "sigma_hat": sigma_hat,
            "E": E_obs,
            "B_used": B_obs,
            "r": r_obs,
            "V_hat_oos": V_hat,
        })

oos_weekly_assets_df = pd.DataFrame(oos_rows).sort_values(["gvkey", "date"]).reset_index(drop=True)
oos_weekly_assets_df

,gvkey,date,sigma_hat,E,B_used,r,V_hat_oos
0,100022,2014-01-03,0.084816,5.055556e+10,1.014480e+11,0.000991,1.519031e+11
1,100022,2014-01-10,0.084816,5.005590e+10,1.014480e+11,0.000963,1.514063e+11
2,100022,2014-01-17,0.084816,5.189801e+10,1.014480e+11,0.000996,1.532450e+11
3,100022,2014-01-24,0.084816,4.934555e+10,1.014480e+11,0.000794,1.507131e+11
4,100022,2014-01-31,0.084816,4.865325e+10,1.014480e+11,0.000371,1.500636e+11
5,100022,2014-02-07,0.084816,4.924321e+10,1.014480e+11,0.000609,1.506294e+11
6,100022,2014-02-14,0.084816,5.177159e+10,1.014480e+11,0.000561,1.531627e+11
7,100022,2014-02-21,0.084816,5.130203e+10,1.014480e+11,0.000639,1.526852e+11
8,100022,2014-02-28,0.084816,5.068800e+10,1.014480e+11,0.000872,1.520476e+11
9,100022,2014-03-07,0.084816,4.927933e+10,1.014480e+11,0.001225,1.506031e+11


In [ ]:
# OOS PD calculation

df = oos_weekly_pds_df.copy()

# Risk-neutral PD (safe to do vectorized as long as inputs are finite)
df["PD_Q_1y"] = merton_pd_rn_1y(
    df["V_hat_oos"].to_numpy(dtype=float),
    df["B_used"].to_numpy(dtype=float),
    df["r"].to_numpy(dtype=float),
    df["sigma_hat"].to_numpy(dtype=float),
)

# Physical PD: only compute where mu_hat_oos is finite (avoid np.any NaN -> all NaN)
df["PD_P_1y"] = np.nan
mask = np.isfinite(df["mu_hat_oos"].to_numpy(dtype=float))

df.loc[mask, "PD_P_1y"] = merton_pd_physical_1y(
    df.loc[mask, "V_hat_oos"].to_numpy(dtype=float),
    df.loc[mask, "B_used"].to_numpy(dtype=float),
    df.loc[mask, "mu_hat_oos"].to_numpy(dtype=float),
    df.loc[mask, "sigma_hat"].to_numpy(dtype=float),
)

# save back + preview
oos_weekly_pds_df = df.sort_values(["gvkey", "date"]).reset_index(drop=True)
oos_weekly_pds_df[["gvkey","date","sigma_hat","mu_hat_oos","V_hat_oos","B_used","r","PD_Q_1y","PD_P_1y"]].head(30)

,gvkey,date,sigma_hat,mu_hat_oos,V_hat_oos,B_used,r,PD_Q_1y,PD_P_1y
0,100022,2014-01-03,0.084816,NaN,1.519031e+11,1.014480e+11,0.000991,1.128432e-06,NaN
1,100022,2014-01-10,0.084816,NaN,1.514063e+11,1.014480e+11,0.000963,1.366075e-06,NaN
2,100022,2014-01-17,0.084816,0.232279,1.532450e+11,1.014480e+11,0.000996,6.734466e-07,2.021663e-14
3,100022,2014-01-24,0.084816,-0.132728,1.507131e+11,1.014480e+11,0.000794,1.794264e-06,1.108011e-03
4,100022,2014-01-31,0.084816,-0.154787,1.500636e+11,1.014480e+11,0.000371,2.347276e-06,2.992129e-03
5,100022,2014-02-07,0.084816,-0.083970,1.506294e+11,1.014480e+11,0.000609,1.871432e-06,1.428329e-04
6,100022,2014-02-14,0.084816,0.075168,1.531627e+11,1.014480e+11,0.000561,7.133368e-07,5.959513e-09
7,100022,2014-02-21,0.084816,0.041747,1.526852e+11,1.014480e+11,0.000639,8.534422e-07,6.820186e-08
8,100022,2014-02-28,0.084816,0.009778,1.520476e+11,1.014480e+11,0.000872,1.075179e-06,6.370263e-07
9,100022,2014-03-07,0.084816,-0.046062,1.506031e+11,1.014480e+11,0.001225,1.825010e-06,2.322042e-05
